In [ ]:
#| default_exp clinical_tutor

# Clinical Tutor

> Core module for using learning context for context-appropriate tutor responses


## Setup

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, List, Optional, Any, Tuple
import os
import requests
import json
import logging
from pathlib import Path
from dotenv import load_dotenv
from wardbuddy.learning_context import LearningContext, setup_logger
import aiohttp

# Load environment variables
load_dotenv()

logger = setup_logger(__name__)

C:\Users\deepa\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Adaptive Clinical Tutor

This module implements:
* Context-aware medical tutoring
* Adaptive feedback generation
* Learning objective management
* Integration with OpenRouter API

In [ ]:
#| export

class ClinicalTutor:
    """
    Adaptive clinical education module that provides context-aware feedback.
    
    Attributes:
        learning_context (LearningContext): Student's learning context
        model (str): LLM model identifier
        api_url (str): OpenRouter API endpoint
    """
    
    def __init__(
        self,
        context_path: Optional[Path] = None,
        model: str = "anthropic/claude-3.5-sonnet"
    ):
        """
        Initialize clinical tutor.
        
        Args:
            context_path: Optional path to saved context
            model: Model identifier for OpenRouter
        """
        self.api_key: str = os.getenv("OPENROUTER_API_KEY")
        if not self.api_key:
            raise ValueError("OpenRouter API key not found")
        
        self.api_url: str = "https://openrouter.ai/api/v1/chat/completions"
        self.model: str = model
        
        self.learning_context = LearningContext(context_path)
        self.context_path = context_path
        
        logger.info(f"Clinical tutor initialized with model: {model}")
        async def _get_completion(
            self,
            messages: List[Dict],
            temperature: float = 0.7
        ) -> str:
            """
            Get completion from OpenRouter API.
            
            Args:
                messages: List of conversation messages
                temperature: Temperature for response generation
                
            Returns:
                str: Model response
            """
            headers = {
                "Authorization": f"Bearer {self.api_key}",
                "Content-Type": "application/json"
            }
            
            data = {
                "model": self.model,
                "messages": messages,
                "temperature": temperature
            }
            
            try:
                response = requests.post(
                    self.api_url,
                    headers=headers,
                    json=data
                )
                response.raise_for_status()
                return response.json()["choices"][0]["message"]["content"]
            except Exception as e:
                logger.error(f"Error in API call: {str(e)}")
                raise

    async def _get_completion(self, messages: List[Dict], temperature: float = 0.7) -> str:
        """Get completion from OpenRouter API.
        
        Args:
            messages: List of conversation messages
            temperature: Temperature for response generation
            
        Returns:
            str: Model response
        """
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
            "HTTP-Referer": "http://localhost:7860",  # Required by OpenRouter
        }
        
        data = {
            "model": self.model,
            "messages": messages,
            "temperature": temperature
        }
        
        try:
            async with aiohttp.ClientSession() as session:
                async with session.post(
                    self.api_url,
                    headers=headers,
                    json=data,
                    timeout=30
                ) as response:
                    response.raise_for_status()
                    result = await response.json()
                    return result["choices"][0]["message"]["content"]
        except Exception as e:
            logger.error(f"Error in API call: {str(e)}")
            raise

    
    def _build_system_prompt(self) -> str:
        """
        Build dynamic system prompt based on context.
        
        Returns:
            str: Contextualized system prompt
        """
        base_prompt = """You are an experienced medical educator providing adaptive feedback 
        and guidance. Consider the student's current learning context, recent cases, and 
        identified areas for improvement when providing feedback. Focus on:
        
        1. Clinical reasoning development
        2. Knowledge integration
        3. Presentation skills
        4. Practical application
        
        Maintain a supportive but challenging teaching style."""
        
        context = self.learning_context.get_context_prompt()
        if context:
            base_prompt += f"\n\nStudent Context:\n{context}"
        
        return base_prompt
    
    async def process_interaction(
        self,
        content: str,
        interaction_type: str = "case_presentation",
        temperature: float = 0.7,
        api_key: Optional[str] = None
    ) -> Dict[str, Any]:
        """
        Process any type of learning interaction with context-aware feedback.
        
        Args:
            content: Student's input
            interaction_type: Type of interaction
            temperature: Temperature for response generation
            api_key: Optional API key override for testing
            
        Returns:
            dict: Processed response with feedback and learning points
        """
        try:
            system_prompt = self._build_system_prompt()
            
            messages = [{
                "role": "system",
                "content": system_prompt
            }, {
                "role": "user",
                "content": f"""Process this {interaction_type}:
                
                {content}
                
                Provide feedback addressing:
                1. Key learning points
                2. Areas for improvement
                3. Relevant connections to recent cases (if any)
                4. Suggested focus areas for future learning
                
                Return your response in JSON format with these keys:
                - feedback: Main feedback message
                - learning_points: List of key learning points
                - gaps: Dict of identified knowledge gaps with confidence scores (0-1)
                - suggestions: List of specific improvement suggestions
                - connections: List of relevant connections to previous cases"""
            }]
            
            if api_key:
                # Use provided API key (for testing)
                original_key = self.api_key
                self.api_key = api_key
                
            try:
                response = await self._get_completion(messages, temperature)
                processed_response = json.loads(response)
            finally:
                if api_key:
                    # Restore original API key
                    self.api_key = original_key
            
            # Update learning context
            self.learning_context.update_from_interaction({
                'case': content,
                'feedback': processed_response['feedback'],
                'gaps': processed_response['gaps'],
                'learning_points': processed_response['learning_points']
            })
            
            # Save updated context if path provided
            if self.context_path:
                self.learning_context.save_context(self.context_path)
            
            return processed_response
            
        except Exception as e:
            logger.error(f"Error processing interaction: {str(e)}")
            return {
                "feedback": "An error occurred processing your input.",
                "learning_points": [],
                "gaps": {},
                "suggestions": ["Please try again or contact support."],
                "connections": []
            }    
    def update_learning_objectives(self, objectives: Dict[str, Any]):
        """
        Update learning objectives in context.
        
        Args:
            objectives: New learning objectives
        """
        self.learning_context.learning_objectives.update(objectives)
        if self.context_path:
            self.learning_context.save_context(self.context_path)
    
    def update_feedback_preferences(self, preferences: Dict[str, float]):
        """
        Update feedback preferences in context.
        
        Args:
            preferences: New feedback preferences
        """
        self.learning_context.feedback_preferences.update(preferences)
        if self.context_path:
            self.learning_context.save_context(self.context_path)

## API Integration

In [ ]:
#| export
class OpenRouterException(Exception):
    """Custom exception for OpenRouter API errors"""
    pass

In [ ]:
#| export
async def safe_api_call(
    url: str,
    headers: Dict[str, str],
    data: Dict[str, Any],
    max_retries: int = 3
) -> Dict:
    """
    Make API call with retry logic and error handling.
    
    Args:
        url: API endpoint
        headers: Request headers
        data: Request data
        max_retries: Maximum retry attempts
        
    Returns:
        dict: API response
        
    Raises:
        OpenRouterException: If API call fails after retries
    """
    for attempt in range(max_retries):
        try:
            response = requests.post(url, headers=headers, json=data)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            if attempt == max_retries - 1:
                raise OpenRouterException(f"API call failed: {str(e)}")
            logger.warning(f"Retry {attempt + 1} after error: {str(e)}")
    
    raise OpenRouterException("Max retries exceeded")

## Tests

In [ ]:
#| hide
async def test_clinical_tutor():
    """Test ClinicalTutor functionality"""
    if not os.getenv("OPENROUTER_API_KEY"):
        print("Skipping tests: No API key")
        return
        
    tutor = ClinicalTutor()
    
    # Test case interaction
    test_case = """
    28yo M with chest pain
    - 2 days duration
    - Sharp, pleuritic
    - No fever or cough
    - Vitals stable
    - Clear exam
    A/P: Likely MSK pain
    """
    
    try:
        # Test basic interaction
        response = await tutor.process_interaction(test_case)
        assert all(k in response for k in ['feedback', 'learning_points', 'gaps'])
        print("Basic interaction test passed")
        
        # Test context integration
        tutor.update_learning_objectives({
            'pulmonology': 'Focus on chest pain differentiation'
        })
        
        response2 = await tutor.process_interaction(test_case)
        assert 'pulmonology' in tutor._build_system_prompt()
        print("Context integration test passed")
        
        # Test error handling with invalid API key
        error_response = await tutor.process_interaction("test", api_key="invalid_key")
        assert "error" in error_response.get("feedback", "").lower()
        assert len(error_response.get("learning_points", [])) == 0
        assert len(error_response.get("gaps", {})) == 0
        print("Error handling test passed")
            
    except Exception as e:
        print(f"Test failed: {str(e)}")
        raise
    
    print("All clinical tutor tests passed!")

# Run tests only when module is run directly
if __name__ == "__main__":
    import asyncio
    if not asyncio.get_event_loop().is_running():
        asyncio.run(test_clinical_tutor())
    else:
        print("Tests skipped - event loop already running")

RuntimeError: asyncio.run() cannot be called from a running event loop